In [17]:
import pandas as pd
import numpy as np

# Loading stations information

In [29]:
stations=pd.read_csv("berlin_measure_stations.csv",sep=";")
stations.drop(["Unnamed: 5","Unnamed: 6","Unnamed: 7"],axis=1,inplace=True)
stations.drop(17,axis=0,inplace=True)
stations.head()

,station,type,nördliche Breite,östliche Länge,Code
0,010 Wedding,Hintergrund,"52° 32' 34,9476""","13° 20' 57,5736""",mc010
1,018 Schöneberg,Hintergrund,"52° 29' 8,93""","13° 20' 55,59""",mc018
2,027 Marienfelde,Stadtrand,"52° 23' 54,26""","13° 22' 5,17""",mc027
3,032 Grunewald,Stadtrand,"52° 28' 23,49""","13° 13' 30,52""",mc032
4,042 Neukölln,Hintergrund,"52° 29' 21,98""","13° 25' 51,08""",mc042


# Downloading Berlin air quality data

In [19]:
dates=[["01.01.2017","01.12.2017"],["02.12.2017","20.11.2018"],["21.11.2018","06.11.2019"]]

In [24]:

new_df=pd.DataFrame()

for code in stations["Code"]:
    for day in dates:
        csv_downloadlink = f"https://luftdaten.berlin.de/station/{code}.csv?group=pollution&period=1h&timespan=custom&start%5Bdate%5D={day[0]}&start%5Bhour%5D=00&end%5Bdate%5D={day[1]}&end%5Bhour%5D=23"
        district = pd.read_csv(csv_downloadlink, sep=";", header=[1,2])
        district = district.drop([district.index[0]])
        
        district.columns = [f'{i}{j}' for i, j in district.columns]
        district = district.rename(columns={"Feinstaub (PM10)µg/m³" : "PM10",
                                        "Kohlenmonoxidmg/m³" : "CO",
                                        "Stickoxideµg/m³": "NOx",
                                        "Stickstoffdioxidµg/m³" : "NO_2", 
                                        "Ozonµg/m³" : "O_3",
                                        "Stickstoffmonoxidµg/m³" : "NO",
                                        "Schwefeldioxidµg/m³" : "SO_2", 
                                        "Ortschaft" : "city",
                                        "MesskomponenteEinheit" : "date"})
        if "CO" not in list(district.columns):
            district=district.assign(CO=np.nan)
        if "SO_2" not in list(district.columns):
            district=district.assign(SO_2=np.nan)
        if "PM10" not in list(district.columns):
            district=district.assign(PM10=np.nan)
        if "O_3" not in list(district.columns):
            district=district.assign(O_3=np.nan)
        if "NO_2" not in list(district.columns):
            district=district.assign(NO_2=np.nan)
        if "NOx" not in list(district.columns):
            district=district.assign(NOx=np.nan)
        if "NO" not in list(district.columns):
            district=district.assign(NO=np.nan)
        cols_to_keep=["date","CO","NO_2","NOx", "O_3", "PM10", "SO_2","NO"]
        to_drop=[col for col in district.columns if col not in cols_to_keep]
        district.drop(to_drop,axis=1,inplace=True)
        district=district[["date","CO","NO_2","NOx", "O_3", "PM10", "SO_2","NO"]]
        district=district.assign(Code=code)
        new_df=new_df.append(district,ignore_index=True)

In [39]:
new_df

,date,CO,NO_2,NOx,O_3,PM10,SO_2,NO,Code
0,01.01.2017 00:00,NaN,28,34,31,43,NaN,4,mc010
1,01.01.2017 01:00,NaN,48,76,8,185,NaN,19,mc010
2,01.01.2017 02:00,NaN,37,45,19,104,NaN,6,mc010
3,01.01.2017 03:00,NaN,NaN,NaN,22,67,NaN,NaN,mc010
4,01.01.2017 04:00,NaN,NaN,NaN,32,31,NaN,NaN,mc010
...,...,...,...,...,...,...,...,...,...
416162,06.11.2019 19:00,NaN,44,81,NaN,NaN,NaN,24,mc282
416163,06.11.2019 20:00,NaN,42,81,NaN,NaN,NaN,25,mc282
416164,06.11.2019 21:00,NaN,34,44,NaN,NaN,NaN,6,mc282
416165,06.11.2019 22:00,NaN,27,31,NaN,NaN,NaN,2,mc282


# Checking types

## Stations dataframe

In [30]:
stations.dtypes

station             object
type                object
nördliche Breite    object
östliche Länge      object
Code                object
dtype: object

## Pollution dataframe

In [56]:
pollution=new_df.copy()
pollution.dtypes

date    object
CO      object
NO_2    object
NOx     object
O_3     object
PM10    object
SO_2    object
NO      object
Code    object
dtype: object

In [58]:
pollution["date"]=pollution["date"].str.replace(".","/")

In [59]:
# Changing date to datetime
pollution["date"]=pd.to_datetime(pollution["date"],dayfirst=True,format="%d/%m/%Y %H:00")
pollution["date"]

0        2017-01-01 00:00:00
1        2017-01-01 01:00:00
2        2017-01-01 02:00:00
3        2017-01-01 03:00:00
4        2017-01-01 04:00:00
                 ...        
416162   2019-11-06 19:00:00
416163   2019-11-06 20:00:00
416164   2019-11-06 21:00:00
416165   2019-11-06 22:00:00
416166   2019-11-06 23:00:00
Name: date, Length: 416167, dtype: datetime64[ns]

In [60]:
# Changing pollutants to int
pollution=pollution.astype({"CO":"float","NO_2":"float","NOx":"float", "O_3":"float", "PM10":"float", "SO_2":"float","NO":"float"})

In [61]:
pollution.dtypes

date    datetime64[ns]
CO             float64
NO_2           float64
NOx            float64
O_3            float64
PM10           float64
SO_2           float64
NO             float64
Code            object
dtype: object

In [62]:
pollution.head()

,date,CO,NO_2,NOx,O_3,PM10,SO_2,NO,Code
0,2017-01-01 00:00:00,NaN,28.0,34.0,31.0,43.0,NaN,4.0,mc010
1,2017-01-01 01:00:00,NaN,48.0,76.0,8.0,185.0,NaN,19.0,mc010
2,2017-01-01 02:00:00,NaN,37.0,45.0,19.0,104.0,NaN,6.0,mc010
3,2017-01-01 03:00:00,NaN,NaN,NaN,22.0,67.0,NaN,NaN,mc010
4,2017-01-01 04:00:00,NaN,NaN,NaN,32.0,31.0,NaN,NaN,mc010


# Exporting files as pickle

In [63]:
stations.to_pickle("stations.pkl")
pollution.to_pickle("pollution.pkl")